In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
# 1. Data Preparation and Loading
# create **known** parameters
weight = 0.7
bias = 0.3

# create 
start = 0
end = 1
steps = 0.02
X = torch.arange(start, end, steps).unsqueeze(dim=1) # add 1 dims

y = weight * X + bias
X[:10]
y[:10]

tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260]])

In [3]:
## Spliting data into training and test sets    
# traing, validation and test set

# creating training set
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

In [4]:
#a# Visualize, Visualize, Visualize
import matplotlib.pyplot as plt

In [5]:
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14})
  plt.show()

In [6]:
# plot_predictions()

#### Build a model
Create linear regression model class
 building classes that can use the following link:
https://realpython.com/python-classes/


In [7]:
# what model does:
# start with random weights and bias
# look at the training data and adjust weights and bias to better represent the data

# through two main algorithms:
# 1. Gradient Descent
# 2. Backpropagation

In [8]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # initialize model parameters (weights and bias)
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad = True,
                                                dtype = torch.float))
        self.bias = nn.Parameter(torch.randn(1,
                                             requires_grad = True,
                                             dtype = torch.float))
        # Forward method to define the computation in the model
    def forward(self,x: torch.Tensor) -> torch.Tensor: # x is input data
        return self.weights * x + self.bias

In [9]:
# Through two main algorithms
# 1. Gradient descent
# 2. Backpropagation

### Pytorch model biulding essential 

__torch.nn__ - contains all of the buildings for computational graphs

__torch.nn.Parameter__ - what parameters should our model try and learn

__torch.nn.Module__ - The base class for all neural network modules

__torch.optim__ - this where the optimizers in PyTorch live, help with gradient descent

__def forward()__ - All nn.Module subclasses require you to overwrite forward()



In [10]:
model_0 = LinearRegressionModel()

## access to the model parameters
model_0.parameters()

torch.manual_seed(42)   # reproducibility of the model parameters

list(model_0.parameters())

## the better way to access model parameters
model_0.state_dict()


OrderedDict([('weights', tensor([-0.6131])), ('bias', tensor([2.0872]))])

In [11]:
### ideal parameters are predefined earlier
# weight = 0.7
# bias = 0.3

# the model task is to start from random parameters and learn the ideal parameters
# making prediction using `torch.inference_mode()`


In [12]:
X_test, y_test

(tensor([[0.8000],
         [0.8200],
         [0.8400],
         [0.8600],
         [0.8800],
         [0.9000],
         [0.9200],
         [0.9400],
         [0.9600],
         [0.9800]]),
 tensor([[0.8600],
         [0.8740],
         [0.8880],
         [0.9020],
         [0.9160],
         [0.9300],
         [0.9440],
         [0.9580],
         [0.9720],
         [0.9860]]))

In [13]:
## how model predicts `y_test` based on `X_test`
with torch.inference_mode():   # this disables gradient tracking, make pytorch use less memory
    y_preds = model_0(X_test)
y_preds

## alternatively one can use
# with torch.no_grad():
#     y_preds = model_0(X_test)
# y_preds


tensor([[1.5967],
        [1.5845],
        [1.5722],
        [1.5600],
        [1.5477],
        [1.5354],
        [1.5232],
        [1.5109],
        [1.4987],
        [1.4864]])

In [14]:
type(X_train.numpy()), type(y_train.numpy()), type(X_test.numpy()), type(y_test.numpy())
X_test.numpy(), y_test.shape

(array([[0.8       ],
        [0.82      ],
        [0.84000003],
        [0.86      ],
        [0.88      ],
        [0.90000004],
        [0.92      ],
        [0.94      ],
        [0.96      ],
        [0.98      ]], dtype=float32),
 torch.Size([10, 1]))

In [ ]:
## lets compare predictions with ideal values
import numpy as np
X = (X_train.numpy())
y = (y_train.numpy())
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))

plt.scatter(X, y, label="Training Values")
# plt.scatter(X_test, y_test, label="Ideal Values", c = 'g')
# plt.scatter(X_test, y_preds, label="Model Predictions", c = 'r')
plt.legend()
plt.show()    # poor model! 

: 

### Train model
training model to move from some __unknown__ parms to some __known__ parms. 

one way: use __loss function__ 

* **Loss Function** A func to measure how wrong your models pred to ideal outputs, lower is better

* __Optimizer__: takes into account the loss and adjust model's parms to improve the loss func, where we need
    * A training loop
    * A testing loop

In [15]:
list(model_0.parameters())

[Parameter containing:
 tensor([-0.6131], requires_grad=True),
 Parameter containing:
 tensor([2.0872], requires_grad=True)]

In [16]:
## kind of loss functions
## 1. Mean Absolute Error Loss --> `torch.nn.L1Loss()`          -- l_n = |y - ŷ|
## 2. Mean Squared Error Loss --> `torch.nn.MSELoss()`          -- l_n = (y - ŷ)²
## 3. Cross Entropy Loss --> `torch.nn.CrossEntropyLoss()`      -- l_n = -Σ(y * log(ŷ))

## Building a training loop in PyTorch

0. loop through the data
1. Forward pass  to make preds on data
2. Calculate the loss
3. Optomizer zero grad
4. Loss Backward 
5. Optimizer step (__gradiant descent__) 

In [17]:
## Use MAE loss
loss_fn = nn.L1Loss()

## setup an optimizer (Stochastic Gradient Descent)
optimizer = torch.optim.SGD(params = model_0.parameters(), lr=0.01)   # lr = learning rate

In [18]:

torch.manual_seed(42)
# An epoch is one loop over the entire dataset
epochs = 301
#===============================
### Track model values
epoch_count = []
loss_values = []
test_loss_values = []
# ==============================
### Training 
# 0. loop through the data
for epoch in range(epochs):
    # set the model to training mode
    model_0.train()   # Set the parameters to that require gradients
    # 1. Forward pass
    y_pred = model_0(X_train)

    # 2. calculate the loss
    loss = loss_fn(y_pred, y_train)     # input first, target next
    # print(f'Loss: {loss.item()}')
    # 3. optimize zero gradients
    optimizer.zero_grad()

    # 4. perform backwardpropagation on the loss with respect to the model parameters
    loss.backward()

    # 5. step the optimizer (perform gradient descent)
    optimizer.step()

    #### Testing code
    model_0.eval()    # turn off gradient tracking or equivalently
    with torch.inference_mode():  # turn of gradient tracking
        # 1. do the forward pass
        test_pred = model_0(X_test)
        # 2. calculate the loss
        test_loss = loss_fn(test_pred, y_test)

    if epoch % 10 ==0:
        epoch_count.append(epoch)
        loss_values.append(loss)
        test_loss_values.append(test_loss)

        print(f'Epoch:{epoch} | Loss: {loss} | Test loss: {test_loss}')

    ## print out model state_dict
        print(model_0.state_dict())

Epoch:0 | Loss: 1.2750914096832275 | Test loss: 0.6050930619239807
OrderedDict({'weights': tensor([-0.6170]), 'bias': tensor([2.0772])})
Epoch:10 | Loss: 1.159881591796875 | Test loss: 0.4703831672668457
OrderedDict({'weights': tensor([-0.6560]), 'bias': tensor([1.9772])})
Epoch:20 | Loss: 1.044671654701233 | Test loss: 0.33567333221435547
OrderedDict({'weights': tensor([-0.6950]), 'bias': tensor([1.8772])})
Epoch:30 | Loss: 0.9294617772102356 | Test loss: 0.20096352696418762
OrderedDict({'weights': tensor([-0.7340]), 'bias': tensor([1.7772])})
Epoch:40 | Loss: 0.8142518997192383 | Test loss: 0.08836555480957031
OrderedDict({'weights': tensor([-0.7730]), 'bias': tensor([1.6772])})
Epoch:50 | Loss: 0.6990420818328857 | Test loss: 0.09088452160358429
OrderedDict({'weights': tensor([-0.8120]), 'bias': tensor([1.5772])})
Epoch:60 | Loss: 0.5854200124740601 | Test loss: 0.20147180557250977
OrderedDict({'weights': tensor([-0.8502]), 'bias': tensor([1.4782])})
Epoch:70 | Loss: 0.5029255747795

In [19]:
model_0.state_dict()

OrderedDict([('weights', tensor([-0.6587])), ('bias', tensor([0.8707]))])

In [20]:
## real parameters
weight, bias

(0.7, 0.3)

In [21]:
## how model predicts `y_test` based on `X_test`

with torch.inference_mode():   # this disables gradient tracking, make pytorch use less memory
    y_preds_new = model_0(X_test)

In [24]:
## lets compare predictions with ideal values

import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))

plt.scatter(X_train, y_train, label="Training Values")
plt.scatter(X_test, y_test, label="Ideal Values", c = 'g')
plt.scatter(X_test, y_preds_new, label="Model Predictions", c = 'r')
plt.legend()
plt.show()    # poor model! 

: 

In [22]:
# plot the loss curves
# epoch_count, loss_values, test_loss_values

In [22]:
import numpy as np
# np.array(torch.tensor(loss_values).numpy())
# loss_values = loss_values.detach()
with torch.inference_mode():
    loss_array = torch.tensor(loss_values).numpy()

In [23]:
## Plot the loss curves
import matplotlib.pyplot as plt
plt.plot(epoch_count, loss_array, label='Train Loss')
plt.plot(epoch_count, test_loss_values, label='Test Loss')


: 

#### Saving model in Pytorch
Three main models for saving and Loading model

1. `torch.save()` save in python's pickle format
2. `torch.load()` allows load a saved Pytorch object
3. `torch.Module.load_state_dict()` load a model's saved state dict

In [22]:
#### Saving our Pytorch model
from pathlib import Path
# 1. Create models dict 
MODEL_PATH = Path('models')
MODEL_PATH.mkdir(parents=True, exist_ok=True)
# 2. create model save path
MODEL_NAME = '01_pytorch_001.pth'
MODEL_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model
torch.save(model_0.state_dict(), MODEL_PATH)

In [24]:
# ls -l models

In [28]:
# How to load a pytorch model
# saved a model_0.state_dict() earlier
# have to instantiate a new instance of our model class
loaded_model_0 = LinearRegressionModel()
loaded_model_0.load_state_dict(torch.load("models/01_pytorch_001.pth"))    
loaded_model_0.state_dict()

OrderedDict([('weights', tensor([-0.6587])), ('bias', tensor([0.8707]))])